In [1]:
pip install flask


Note: you may need to restart the kernel to use updated packages.


In [2]:
contact_management_system/
│
├── app.py             # Main app (Flask)
├── templates/         # HTML files
│   ├── index.html     # Home page (Login/Registration)
│   ├── dashboard.html # User dashboard (Contact management)
│   └── add_contact.html # Add new contact
└── static/            # CSS and JS files (optional for styling)
    └── style.css


SyntaxError: invalid character '│' (U+2502) (1606571623.py, line 2)

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
import sqlite3
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Change to a secure key!

# Database setup
def init_db():
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    username TEXT UNIQUE,
                    password TEXT,
                    email TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS contacts (
                    id INTEGER PRIMARY KEY,
                    user_id INTEGER,
                    name TEXT,
                    phone TEXT,
                    email TEXT,
                    FOREIGN KEY(user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()

# Route for the homepage (login/registration)
@app.route('/')
def index():
    return render_template('index.html')

# Route to handle user registration
@app.route('/register', methods=['POST'])
def register():
    username = request.form['username']
    password = request.form['password']
    email = request.form['email']

    # Hash the password
    hashed_password = generate_password_hash(password)

    # Insert user into the database
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('INSERT INTO users (username, password, email) VALUES (?, ?, ?)', 
              (username, hashed_password, email))
    conn.commit()
    conn.close()

    return redirect(url_for('index'))

# Route to handle user login
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']

    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM users WHERE username = ?', (username,))
    user = c.fetchone()

    if user and check_password_hash(user[2], password):  # user[2] is the hashed password
        session['user_id'] = user[0]
        return redirect(url_for('dashboard'))
    return 'Login Failed, please try again.'

# Dashboard (User homepage after login)
@app.route('/dashboard')
def dashboard():
    if 'user_id' not in session:
        return redirect(url_for('index'))

    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM contacts WHERE user_id = ?', (session['user_id'],))
    contacts = c.fetchall()
    conn.close()
    return render_template('dashboard.html', contacts=contacts)

# Add a new contact
@app.route('/add_contact', methods=['GET', 'POST'])
def add_contact():
    if 'user_id' not in session:
        return redirect(url_for('index'))

    if request.method == 'POST':
        name = request.form['name']
        phone = request.form['phone']
        email = request.form['email']

        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute('INSERT INTO contacts (user_id, name, phone, email) VALUES (?, ?, ?, ?)',
                  (session['user_id'], name, phone, email))
        conn.commit()
        conn.close()

        return redirect(url_for('dashboard'))
    return render_template('add_contact.html')

if __name__ == '__main__':
    init_db()  # Initialize the database
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Contact Management</title>
</head>
<body>
    <h2>Login</h2>
    <form method="POST" action="/login">
        <label for="username">Username</label>
        <input type="text" id="username" name="username" required><br>
        <label for="password">Password</label>
        <input type="password" id="password" name="password" required><br>
        <button type="submit">Login</button>
    </form>

    <h3>Not a member? Register below!</h3>
    <form method="POST" action="/register">
        <label for="username">Username</label>
        <input type="text" id="username" name="username" required><br>
        <label for="email">Email</label>
        <input type="email" id="email" name="email" required><br>
        <label for="password">Password</label>
        <input type="password" id="password" name="password" required><br>
        <button type="submit">Register</button>
    </form>
</body>
</html>


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Your Dashboard</title>
</head>
<body>
    <h2>Your Contacts</h2>
    <a href="/add_contact">Add New Contact</a>
    <ul>
        {% for contact in contacts %}
            <li>{{ contact[2] }} - {{ contact[3] }} - {{ contact[4] }}</li>
        {% endfor %}
    </ul>
</body>
</html>


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Add Contact</title>
</head>
<body>
    <h2>Add a New Contact</h2>
    <form method="POST">
        <label for="name">Name</label>
        <input type="text" id="name" name="name" required><br>
        <label for="phone">Phone</label>
        <input type="text" id="phone" name="phone" required><br>
        <label for="email">Email</label>
        <input type="email" id="email" name="email" required><br>
        <button type="submit">Add Contact</button>
    </form>
</body>
</html>


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session, jsonify
import sqlite3
import jwt
import datetime
from werkzeug.security import generate_password_hash, check_password_hash
from functools import wraps

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Change to a secure key!

# Secret key for JWT
SECRET_KEY = "your_jwt_secret_key_here"

# Database setup (no change)
def init_db():
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    username TEXT UNIQUE,
                    password TEXT,
                    email TEXT)''')
    c.execute('''CREATE TABLE IF NOT EXISTS contacts (
                    id INTEGER PRIMARY KEY,
                    user_id INTEGER,
                    name TEXT,
                    phone TEXT,
                    email TEXT,
                    FOREIGN KEY(user_id) REFERENCES users(id))''')
    conn.commit()
    conn.close()

# Helper function to decode JWT token
def token_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        token = request.headers.get('Authorization')
        if not token:
            return jsonify({'message': 'Token is missing!'}), 403

        try:
            data = jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
            current_user_id = data['user_id']
        except:
            return jsonify({'message': 'Token is invalid!'}), 403

        return f(current_user_id, *args, **kwargs)

    return decorated_function

# Route for the homepage (login/registration)
@app.route('/')
def index():
    return render_template('index.html')

# Route to handle user registration
@app.route('/register', methods=['POST'])
def register():
    username = request.form['username']
    password = request.form['password']
    email = request.form['email']

    # Hash the password
    hashed_password = generate_password_hash(password)

    # Insert user into the database
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('INSERT INTO users (username, password, email) VALUES (?, ?, ?)', 
              (username, hashed_password, email))
    conn.commit()
    conn.close()

    return redirect(url_for('index'))

# Route to handle user login
@app.route('/login', methods=['POST'])
def login():
    username = request.form['username']
    password = request.form['password']

    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM users WHERE username = ?', (username,))
    user = c.fetchone()

    if user and check_password_hash(user[2], password):  # user[2] is the hashed password
        token = jwt.encode({
            'user_id': user[0],
            'exp': datetime.datetime.utcnow() + datetime.timedelta(days=1)
        }, SECRET_KEY, algorithm="HS256")
        return jsonify({'token': token})

    return jsonify({'message': 'Login Failed, please try again.'})

# Dashboard (User homepage after login)
@app.route('/dashboard')
@token_required
def dashboard(current_user_id):
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM contacts WHERE user_id = ?', (current_user_id,))
    contacts = c.fetchall()
    conn.close()
    return render_template('dashboard.html', contacts=contacts)

# Add a new contact
@app.route('/add_contact', methods=['GET', 'POST'])
@token_required
def add_contact(current_user_id):
    if request.method == 'POST':
        name = request.form['name']
        phone = request.form['phone']
        email = request.form['email']

        conn = sqlite3.connect('contacts.db')
        c = conn.cursor()
        c.execute('INSERT INTO contacts (user_id, name, phone, email) VALUES (?, ?, ?, ?)',
                  (current_user_id, name, phone, email))
        conn.commit()
        conn.close()

        return redirect(url_for('dashboard', current_user_id=current_user_id))
    return render_template('add_contact.html')

# Edit an existing contact
@app.route('/edit_contact/<int:contact_id>', methods=['GET', 'POST'])
@token_required
def edit_contact(current_user_id, contact_id):
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('SELECT * FROM contacts WHERE id = ? AND user_id = ?', (contact_id, current_user_id))
    contact = c.fetchone()

    if not contact:
        return 'Contact not found!', 404

    if request.method == 'POST':
        name = request.form['name']
        phone = request.form['phone']
        email = request.form['email']

        c.execute('UPDATE contacts SET name = ?, phone = ?, email = ? WHERE id = ?',
                  (name, phone, email, contact_id))
        conn.commit()
        conn.close()

        return redirect(url_for('dashboard', current_user_id=current_user_id))
    
    return render_template('edit_contact.html', contact=contact)

# Delete a contact
@app.route('/delete_contact/<int:contact_id>', methods=['POST'])
@token_required
def delete_contact(current_user_id, contact_id):
    conn = sqlite3.connect('contacts.db')
    c = conn.cursor()
    c.execute('DELETE FROM contacts WHERE id = ? AND user_id = ?', (contact_id, current_user_id))
    conn.commit()
    conn.close()
    return redirect(url_for('dashboard', current_user_id=current_user_id))

if __name__ == '__main__':
    init_db()  # Initialize the database
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Your Dashboard</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h2>Your Contacts</h2>
    <a href="/add_contact">Add New Contact</a>
    <ul>
        {% for contact in contacts %}
            <li>
                {{ contact[2] }} - {{ contact[3] }} - {{ contact[4] }} 
                <a href="/edit_contact/{{ contact[0] }}">Edit</a>
                <form action="/delete_contact/{{ contact[0] }}" method="POST" style="display:inline;">
                    <button type="submit">Delete</button>
                </form>
            </li>
        {% endfor %}
    </ul>
</body>
</html>
